In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name()
plt.rc("font",family=font_name)

import matplotlib as mlp
mlp.rcParams['axes.unicode_minus'] = False

import seaborn as sns

plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

# 우리나라 인구 소멸 위기 지역 분석

65세 이상 노인 인구와 20 ~ 39세 여성인구를 비교해서 젊은 여성인구가 노인인구의 절반을 미달할 경우
<한국 고용정보원 - 한국의 지방소멸에 관한 7가지 분석>

### 1. 데이터 준비

+ 통계청 사이트 : https://kosis.kr/index/index.do

In [32]:
pop = pd.read_excel('data/pop_raw_data.xlsx',header=1)

# 결측치 앞내용으로 채우기
pop.fillna(method='pad',inplace=True)

# 컬럼명 변경
pop.rename(columns={'행정구역별(읍면동)(1)':'광역시도',
                   '행정구역별(읍면동)(2)':'시도',
                   '합계':'인구수',
                   '항목':'구분'},inplace=True)

# 행 데이터 변경
pop.loc[pop['구분']=='총인구(명)','구분'] = '합계'
pop.loc[pop['구분']=='총인구_남자(명)','구분'] = '남자'
pop.loc[pop['구분']=='총인구_여자(명)','구분'] = '여지'

# 준간 준간에 있는 '속계' 항목 삭제
pop = pop[(pop['시도'] != '소계')]

pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 6 to 836
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   광역시도    783 non-null    object
 1   시도      783 non-null    object
 2   구분      783 non-null    object
 3   인구수     783 non-null    int64 
 4   20~24세  783 non-null    int64 
 5   25~29세  783 non-null    int64 
 6   30~34세  783 non-null    int64 
 7   35~39세  783 non-null    int64 
 8   40~44세  783 non-null    int64 
 9   45~49세  783 non-null    int64 
 10  50~54세  783 non-null    int64 
 11  55~59세  783 non-null    int64 
 12  60~64세  783 non-null    int64 
 13  65~69세  783 non-null    int64 
 14  70~74세  783 non-null    int64 
 15  75~79세  783 non-null    int64 
 16  80~84세  783 non-null    int64 
 17  85~89세  783 non-null    int64 
 18  90~94세  783 non-null    int64 
 19  95~99세  783 non-null    object
 20  100세이상  783 non-null    object
dtypes: int64(16), object(5)
memory usage: 134.6+ KB


In [29]:
print(np.unique(pop))

TypeError: '<' not supported between instances of 'int' and 'str'

In [34]:
##### 95~99세, 100세 이상의 값들 중 숫자가 아닌 값이 있는지 확인
# pop.loc[pop['90~94세'] == 'X','90~94세'] = 0 
# pop.loc[pop['95~99세'] == 'X','95~99세'] = 0 
# pop.loc[pop['100세이상'] == 'X','100세이상'] = 0 

pop['95~99세'].unique()
pop['100세이상'].unique()

# pd.to_numeric(pop['100세이상'])
#print(pop['100세이상'].str.extract('([^0-9])').value_counts())

array([21, 6, 15, 17, 5, 12, 22, 23, 18, 32, 27, 26, 8, 41, 'X', 38, 10,
       20, 31, 11, 50, 44, 30, 9, 29, 33, 28, 36, 40, 24, 37, 25, 35, 47,
       7, 43, 16, 13, 19, 14, 48, 45, 34, 70, 56, 73, 55, 42, 78, 65, 103,
       85, 77, 57, 88, 75, 39, 51, 59, 54, 80, 71, 46], dtype=object)

In [38]:
##### 5~99세, 100세 이상의 값들 중 x값을 0으로 대체하고 숫자형으로 변환

pop.loc[pop['90~94세'] == 'X','90~94세'] = 0 
pop.loc[pop['95~99세'] == 'X','95~99세'] = 0 
pop.loc[pop['100세이상'] == 'X','100세이상'] = 0 

pop['95~99세'].unique()

pop['95~99세'] = pd.to_numeric(pop['95~99세'])
pop['100세이상'] = pd.to_numeric(pop['100세이상'])

pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783 entries, 6 to 836
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   광역시도    783 non-null    object
 1   시도      783 non-null    object
 2   구분      783 non-null    object
 3   인구수     783 non-null    int64 
 4   20~24세  783 non-null    int64 
 5   25~29세  783 non-null    int64 
 6   30~34세  783 non-null    int64 
 7   35~39세  783 non-null    int64 
 8   40~44세  783 non-null    int64 
 9   45~49세  783 non-null    int64 
 10  50~54세  783 non-null    int64 
 11  55~59세  783 non-null    int64 
 12  60~64세  783 non-null    int64 
 13  65~69세  783 non-null    int64 
 14  70~74세  783 non-null    int64 
 15  75~79세  783 non-null    int64 
 16  80~84세  783 non-null    int64 
 17  85~89세  783 non-null    int64 
 18  90~94세  783 non-null    int64 
 19  95~99세  783 non-null    int64 
 20  100세이상  783 non-null    int64 
 21  20~39세  783 non-null    int64 
dtypes: int64(19), object(3)
me

### 2.인구소멸 위기지역 계산하고 데이터 정리

#### 20~30대의 인구를 알아야 하고 65세 이상 인구수도 알아야 한다.

In [41]:
pop['20~39세'] = pop['20~24세'] + pop['25~29세'] + pop['30~34세'] +pop['35~39세']

pop['65세이상'] = pop['65~69세'] + pop['70~74세'] + pop['75~79세'] + pop['80~84세'] + \
                    pop['85~89세'] + pop['90~94세'] + pop['95~99세'] + pop['100세이상']

pop.head(10)

,광역시도,시도,구분,인구수,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,...,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세이상,20~39세,65세이상
6,서울특별시,종로구,합계,154969,15242,14057,10753,10504,9846,12652,...,7366,6101,5377,3461,1580,565,147,21,50556,24618
7,서울특별시,종로구,남자,74880,6707,7194,5443,5206,4811,6214,...,3527,2718,2329,1416,555,182,45,6,24550,10778
8,서울특별시,종로구,여지,80089,8535,6863,5310,5298,5035,6438,...,3839,3383,3048,2045,1025,383,102,15,26006,13840
9,서울특별시,중구,합계,130957,10762,12057,11034,10933,8810,9926,...,6665,5286,4333,2692,1234,476,92,17,44786,20795
10,서울특별시,중구,남자,63406,4841,6006,5548,5607,4586,4963,...,3200,2370,1835,1029,370,133,26,5,22002,8968
11,서울특별시,중구,여지,67551,5921,6051,5486,5326,4224,4963,...,3465,2916,2498,1663,864,343,66,12,22784,11827
12,서울특별시,용산구,합계,227181,15289,20431,19789,20216,16585,19022,...,10344,8396,7180,4847,2141,725,194,22,75725,33849
13,서울특별시,용산구,남자,110934,6954,10232,10207,10431,8372,9498,...,4838,3739,3052,1852,759,192,62,5,37824,14499
14,서울특별시,용산구,여지,116247,8335,10199,9582,9785,8213,9524,...,5506,4657,4128,2995,1382,533,132,17,37901,19350
15,서울특별시,성동구,합계,299688,22964,26568,24647,26456,21640,24238,...,13760,10870,8810,5273,2241,724,181,23,100635,41882


In [53]:
##### 일부 컬럼만 따로 추출 (광역시도, 시도, 구분, 인구수, 20~39세, 65세이상,)
##### '합계', '남자','여자' 구분정리

#pop_1 = pop[['광역시도', '시도', '구분', '인구수', '20~39세', '65세이상'],index='광역시도']
pop1 = piv

SyntaxError: invalid syntax (<ipython-input-53-356e56c7ae34>, line 4)

+ 인구 소멸 비율 계산
+ 이 비율이 1보다 작으면 인구소멸 위기지역으로 볼 수 있다.

In [55]:
pop1['소멸비율'] = pop['20~39세','여자'] / (pop1['65세이상','합계']/2)
pop1.head()

KeyError: ('20~39세', '여자')

In [ ]:
##### 소멸의기 지역인지 확인 할 수 있는 boolean값 지정

pop1['소멸위기지역'] = pop1['소멸비율'] < 1.0
pop1.head()

In [ ]:
##### 소멸위기 지역을 리스트로 출력
#1
pop_list = pop.loc[pop['소멸위기지역']==True,'시도']
#2
print(pop1[pop1['소멸위기지역']==True].index.get_level_values(1))

In [56]:
##### 시각화를 위해 모든 컬럼은 하나의 컬럼으로 합치고 현재 다중인덱스도 컬럼으로 조정한다.

pop1.reset_index(inplace=True)
pop1.head()

In [60]:
pop1.columns.get_level_values(0)
pop.columns.get_level_values(0)[0]


'광역시도'

In [ ]:
pop1.columns = [pop1.columns.get_level_values(0)[n] + \
                 pop1.columns.get_level_values(1)[n]
                 for n in range(len(pop1.columns.get_level_values(0)))]

In [ ]:
pop1.info()

### 3.

+ 광역시가 아니면서 구를 가지고 있는 시와 그 행정구를 dict형으로 선언
+ 광역시도에 있는 이름의 끝 세글자가 '광역시', '특별시','자치시'로 끝나지 않으면 일반 시 혹은 군으로 본다.
+ 그 속에서 강원도와 경상남도에는 동일한 이름을 가진 '고성군'이 있어서 그것을 처리
+ 일반 시인데 구를 가지응 경우에 대해 대응한다.
+ 세종특별자치시를 그냥 '세종'으로 처리하고 나머지 광역시도에서 앞 두 글자(서울특별시)와 시도에서 두글자인 경우 모두, 아니면 앞 글자만 선택해서 고유 ID작성

In [ ]:
tmp_gu_dict = {'수원':['장안구', '권선구', '팔달구', '영통구'], 
                       '성남':['수정구', '중원구', '분당구'], 
                       '안양':['만안구', '동안구'], 
                       '안산':['상록구', '단원구'], 
                       '고양':['덕양구', '일산동구', '일산서구'], 
                       '용인':['처인구', '기흥구', '수지구'], 
                       '청주':['상당구', '서원구', '흥덕구', '청원구'], 
                       '천안':['동남구', '서북구'], 
                       '전주':['완산구', '덕진구'], 
                       '포항':['남구', '북구'], 
                       '창원':['의창구', '성산구', '진해구', '마산합포구', '마산회원구'], 
                       '부천':['오정구', '원미구', '소사구']}

si _name = [None] * len(pop1)

In [4]:
for n in pop1.index:
    if pop1['광역시도'][n][-3:] not in ['광역시','특별시','자치시']:
        if pop1['시도'][n][:-1] == '고성' and pop1['광역시도'][n] == '강원도':
            si_name[n] = '고성(강원)
        elif pop1['시도'][n][:-1] == '고성'and pop1['광역시도'][n] == '경상남도':
            si_name[n] = '고성(경남)'
        else:
            si_name[n] = pop1['시도'][n][:-1]
            
        for key, values in tmp_gu_dict.items():
            if pop1['시도'][n] in values:
                if len(pop1['시도'][n]) == 2:
                    si_name[n] = keys + ' ' + pop1['시도'][n]
                elif pop1['시도'][n] in ['마산합포구','마산회원구']:
                    si_name[n] = keys + ' ' + pop1['시도'][n][2:-1]
                else:
                    si_name[n] = keys + ' ' + pop1['시도'][n][:-1]
                    
    elif pop1['광역시도'][n] == '세종특별자치시':
        si_name[n] = '세종'
    else:
        if len(pop1['시도'][n]) == 2:
            si_name[n] = pop1['광역시도'][n][:2] + ' ' + pop1['시도'][n]
        else:
            si_name[n] = pop1['광역시도'][n][:2] + ' ' + pop1['시도'][n][:-1]

SyntaxError: EOL while scanning string literal (<ipython-input-4-8b19ce9fff9a>, line 4)

In [3]:
pop1['ID'] = si_name

NameError: name 'si_name' is not defined

In [ ]:
##### 크게 의미 없는 컬럼 삭제

del pop1['20~39세 남자']
del pop1['65세이상남자']
del pop1['65세이상여자']

In [ ]:
pop1.head()

### 4. 지도 시각화

In [5]:
import folium
import json

In [ ]:
pop2 = pop1.set_index('ID')
pop2.head()

In [6]:
geo_path = 'data/map/skorea_municipalities_geo_simple_update.json'
geo_str = json.load(open(geo_path, encoding = 'utf-8'))

map = folium.Map(location=[36.2002,127.054], zoom_start=7)
map.choropleth(geo_data=geo_str, data=pop2['인구수합계'],
              columns=[pop2.index,pop2['인구수합계']],
              key_on='feature.id',
              fill_color='YlGnBu')

NameError: name 'pop2' is not defined